The politeness model adapted from Berkem (past MSc student working on related work It is a pre-trained transformer model. I use the model he had pre-trained.

In [ ]:
! pip install pandas==1.3.0
!pip install transformers
!pip install torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Train regresh

# Run script

In [ ]:
# polite script

from transformers.file_utils import is_tf_available, is_torch_available
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import AutoConfig
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

class SimpleDataset:
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts
    
    def __len__(self):
        return len(self.tokenized_texts["input_ids"])
    
    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.tokenized_texts.items()}


def compute_metrics_for_regression(eval_pred):
      logits, labels = eval_pred

      labels = labels.reshape(-1, 1)
      
      print("Logits:", logits[0:5])
      print("Labels:", labels[0:5])


      mse = mean_squared_error(labels, logits)
      rmse = mean_squared_error(labels, logits, squared=False)
      mae = mean_absolute_error(labels, logits)
      r2 = r2_score(labels, logits)

      single_squared_errors = ((logits - labels).flatten()**2).tolist()
      accuracy = sum([1 for e in single_squared_errors if e < 0.25]) / len(single_squared_errors)
      
      return {"mse": mse, "rmse": rmse, "mae": mae, "r2": r2, "accuracy": accuracy}

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

def load_politeness_model(user_feeds):
    device = torch.device("cuda")
    ## Scoring each tweet based on politeness 
    class SimpleDataset:
        def __init__(self, tokenized_texts):
            self.tokenized_texts = tokenized_texts
        
        def __len__(self):
            return len(self.tokenized_texts["input_ids"])
        
        def __getitem__(self, idx):
            return {k: v[idx] for k, v in self.tokenized_texts.items()}

    model_name = 'roberta-base'
    print('loaded politeness')
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    with torch.no_grad():
        model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Colab Notebooks/final_year_project/checkpoint-52500/').to(device)

    tweets = df["grammartext"].astype(str).values.tolist()

    test_encodings = tokenizer(tweets , truncation=True, padding=True, max_length=256)
    test_dataset = SimpleDataset(test_encodings)       

    trainer = Trainer(model=model)
    predictions = trainer.predict(test_dataset)
    
    return predictions[0]



In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/final_year_project/run_historic_features/1/historic_retweeter_features_msc_FINAL_NEW_v3.csv')

import torch
polite_out = load_politeness_model(df)
df['politeness'] = polite_out

save_path = '/content/drive/MyDrive/Colab Notebooks/final_year_project/run_historic_features/2/historic_retweeter_features_msc_FINAL_NEW_withpoliteness_v3.csv'

df.to_csv(save_path)

In [ ]:
df.columns

In [ ]:
if 1435553820382269441 in df.id.values:
    print("yes")

yes
